2. Fine-tuning BERTopic

In [6]:
from bertopic import BERTopic
from umap import UMAP
import pandas as pd


In [7]:
# Read CSV File
df = pd.read_csv("/Users/gresasmolica/Downloads/processed_data_congress.csv")

# keep only the text column

df = df[['processed_text_v2']]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   processed_text_v2  52 non-null     object
dtypes: object(1)
memory usage: 544.0+ bytes


In [8]:
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    metric='cosine',
    min_dist=0.0,
    random_state=42
)

topic_model = BERTopic(
    language="english",
    umap_model=umap_model,
    calculate_probabilities=True
)

topics, probabilities = topic_model.fit_transform(df['processed_text_v2'])

In [9]:
# get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,28,0_state_ukraine_russia_senator,"[state, ukraine, russia, senator, would, count...","[['opening', 'statement', 'hon', 'robert', 'me..."
1,1,24,1_mr_thank_think_ukraine,"[mr, thank, think, ukraine, know, would, state...","[['subcommittee', 'met', 'pursuant', 'notice',..."


In [10]:
topic_model.visualize_barchart(top_n_topics=5)

In [11]:
topic_model.visualize_term_rank()

In [13]:
topic_model.visualize_hierarchy()